In [419]:
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [420]:
train = pd.read_csv('datasets/events_sequence_train.csv')
test = pd.read_csv('datasets/events_sequence_test.csv')

In [467]:
datepidu = train['Date'].unique()
pidus = train['PID'].unique()
grad = dict()
for pid in pidus:
    grado = dict()
    for date in datepidu:
        dole = len(train[(train['PID'] == pid) & (train['Date'] == date)])
        if dole > 0:
            grado[date] = (30.0/dole)
    grad[pid] = grado

"pidus = train['PID'].unique()\ngrad = dict()\nfor pid in pidus:\n    grado = dict()\n    for date in datepidu:\n        dole = len(train[(train['PID'] == pid) & (train['Date'] == date)])\n        if dole > 0:\n            grado[date] = (30.0/dole)\n    grad[pid] = grado"

In [550]:
cur = 1
alist = []
alist.append(1)
for pidprev, pidnow, dateprev, datenow in zip(train['PID'].values[:-1], train['PID'].values[1:], train['Date'].values[:-1], train['Date'].values[1:]):
    if pidnow == pidprev:
        cur += 1
    else:
        cur = 1
    alist.append(cur)
train['Day'] = pd.Series(alist)

In [605]:
train['Day'] = pd.Series([15]*len(train))

In [673]:
def yora(date, day):
    '''if date/100 < 2012:
        return 1
    if (date/100) > 2013:
        return 10
    return 2
    #return ((1.5*((date%100)-1))/11) + 1'''
    #year = (date/100)%10
    #return (((year-1)*12 + date%100)**2)*(year**2)
    #return (year-1)*3600 + (date%100)*100 + day*100
    year = int(date)/100
    month = int(date) % 100
    week = int((month-1)*30 + round(day))/7
    if year < 2012:
        return 1
    elif year > 2012 and year < 2013:
        return 1.5
    elif month <= 2:
        return 10
    elif month <= 6:
        return month
    else:
        return (7 + (week-25))

In [674]:
# train['Month'] = pd.Series([(yora(date)*((((date/100)%10 - 1)*12)+date%100) for date in train['Date'].values])
train['Month'] = pd.Series([yora(date, day) for date, day in zip(train['Date'].values, train['Day'].values)])

In [667]:
train.tail(10)

,PID,Date,Event,Day,Month
766777,1022024,201308,8004,15,14.142857
766778,1022024,201308,8006,15,14.142857
766779,1022024,201308,8443,15,14.142857
766780,1022024,201308,8444,15,14.142857
766781,1022024,201308,8502,15,14.142857
766782,1022024,201308,9462,15,14.142857
766783,1022024,201308,9406,15,14.142857
766784,1022024,201308,9436,15,14.142857
766785,1022024,201308,9472,15,14.142857
766786,1022024,201308,9426,15,14.142857


In [675]:
#import random
taking = 6
predictions = []
percentage = 0.75
for pid, i in zip(test['PID'].values, range(len(test['PID'].values))):
    # This 1st line here
    cur = train[(train['PID'] == pid) & (train['Date']/100 > 2013) & (train['Date']%100 >= 6)]
    # cur.sort_values(['Date'], ascending=False, inplace=True)
    # cur = cur.head(min(100, len(cur)))
    # Multiplying by month
    #if len(cur) < 10:
    if len(cur['Event'].unique()) < taking:
        cur = train[(train['PID'] == pid) & (train['Date']/100 > 2013)]
    #if len(cur) < 10:
    if len(cur['Event'].unique()) < taking:
        cur = train[(train['PID'] == pid) & (train['Date']/100 > 2012)]
    #if len(cur) < 10:
    if len(cur['Event'].unique()) < taking:
        cur = train[(train['PID'] == pid)]
    #cur = train[(train['PID'] == pid)].sort_values('Date', ascending=False).head(int(round(percentage*len(train[(train['PID'] == pid)]))))
    #cur.sort_values('Date', ascending=True, inplace=True)
    #cur['Month'] = pd.Series([i for month, i in zip(cur['Month'].values, range(len(cur['Month'].values)))])
    cur = cur.groupby('Event').sum().sort_values('Month', ascending=False).head(10)
    cur1 = cur.index.values
    #freqs = cur['Month'].values
    #probs = [freq/sum(cur['Month'].values) for freq in cur['Month'].values]
    #cur = np.random.choice(cur1, size=10, p=probs)
    cur = cur.index.values[:taking]
    cur = list(cur)
    if len(cur) < taking:
        cur = cur + cur[:taking-len(cur)]
    cur = cur+cur[:10%taking]
    curo = cur
    #random.shuffle(cur)
    predictions.append(curo)
    """curro = [0]*10
    for i in range(10):
        ind = np.random.choice(list(range(len(cur1))), size=1, p=probs)[0]
        freqs[ind] += 13
        probs = [freq/sum(freqs) for freq in freqs]
        curro[i] = cur1[ind]
    # Can apply probability here too
    predictions.append(curro)"""

In [676]:
print(len(predictions))
predictions = np.array(predictions)
predictions.shape

3000


(3000, 10)

In [677]:
for i in range(10):
    test['Event'+str(i+1)] = pd.Series(predictions[:, i])
test.head()

,PID,Event1,Event2,Event3,Event4,Event5,Event6,Event7,Event8,Event9,Event10
0,1028890,2214,8501,8561,3258,7087,3641,2214,8501,8561,3258
1,1021443,2773,2189,8100,8708,V726,5815,2773,2189,8100,8708
2,1016194,2735,2334,3417,2446,3273,2533,2735,2334,3417,2446
3,1017298,3614,1975,2706,3131,2657,2533,3614,1975,2706,3131
4,1006098,2687,3194,7109,3641,2674,8502,2687,3194,7109,3641


In [678]:
df = test
df.to_csv('datasets/events_sequence_predictions.csv', index=False)